# Извлечение ключевых слов

## Корпус

В корпус вошли пять коротких новостей с сайта газеты "Завтра" (http://zavtra.ru/). Ключевые слова представлены на сайте в виде тегов. (К сожалению, не знаю, как именно они расставляют теги).

В строке с ключевыми словами уже есть ключевые слова, размеченные мной вручную (после символа ";").


In [11]:
import os

corpus = [i in os.listdir("corpus") if not i.endswith('.conllu')


In [79]:
tags = {}
for fn in corpus:
    meta_info = {}
    with open(os.path.join("corpus", fn), 'r', encoding='utf-8') as f:
        meta = f.readlines()[:3]
        for line in meta:
            title, info = line.strip('\n').split('\t')
            meta_info[title] = info
    tags[fn] = meta_info

### Сравнение моей разметки и исходной

Так себе сходимся, особенно на совсем маленьких текстах.

In [187]:
all_true_their = []
all_true_mine = []
for inf in tags:
    print(inf)
    before, after = tags[inf]['Ключевые слова'].split(';')
    their = set(before.split(', '))
    all_true_their.append(their)
    
    mine = set(after.split(', '))
    all_true_mine.append(mine)
    print("Есть и у меня, и в исходной:", their.intersection(mine))
    print("Есть в исходной, нет в моей:", their - mine)
    print("Нет в моей, есть в исходной:", mine - their)
    

1.txt
Есть и у меня, и в исходной: {'музыка'}
Есть в исходной, нет в моей: {'школа', 'минкульт'}
Нет в моей, есть в исходной: {'норматив', 'школьник', 'министерство культуры'}
2.txt
Есть и у меня, и в исходной: {'вера', 'выставка', 'художник'}
Есть в исходной, нет в моей: {'русская история', 'победа', 'живопись', 'родина', 'москва'}
Нет в моей, есть в исходной: {'союз художников', 'Россия'}
3.txt
Есть и у меня, и в исходной: {'концерт'}
Есть в исходной, нет в моей: {'рок', 'музыка'}
Нет в моей, есть в исходной: {'втб арена', 'scorpions'}
4.txt
Есть и у меня, и в исходной: {'сердюков', 'васильева', 'выставка', 'современное искусство'}
Есть в исходной, нет в моей: set()
Нет в моей, есть в исходной: {'художник', 'EVA', 'музей востока', 'министерство обороны', 'культура'}
5.txt
Есть и у меня, и в исходной: {'экономика', 'поколение', 'миллениалы', 'сша', 'здоровье'}
Есть в исходной, нет в моей: {'суицид'}
Нет в моей, есть в исходной: {'молодой возраст'}


### Разметка корпуса

In [67]:
comm = """C:\\Users\\Eiko\\PycharmProjects\\NLP_19_20\\udpipe-1.2.0-bin\\bin-win64\\udpipe.exe --input horizontal --output conllu \
--tokenize --tag --parse \
C:\\Users\\Eiko\\PycharmProjects\\NLP_19_20\\russian-syntagrus-ud-2.4-190531.udpipe \
< C:\\Users\\Eiko\\PycharmProjects\\NLP_19_20\\hw1_key_words\\corpus\{} > C:\\Users\\Eiko\\PycharmProjects\\NLP_19_20\\hw1_key_words\\corpus\{}.conllu"""


In [72]:
import subprocess

for filename in corpus:
    commandline = comm.format(filename, filename[:-4])
    try:
        output = subprocess.check_output(
            commandline, stderr=subprocess.STDOUT, shell=True, timeout=1000,
            universal_newlines=True)
    except subprocess.CalledProcessError as exc:
        print("Status : FAIL", exc.returncode, exc.output)
    else:
        print("Output: \n{}\n".format(output))


Output: 
Loading UDPipe model: done.


Output: 
Loading UDPipe model: done.


Output: 
Loading UDPipe model: done.


Output: 
Loading UDPipe model: done.


Output: 
Loading UDPipe model: done.




## Извлечение ключевых слов

In [175]:
# Препроцессинг: лемматизация текстов

import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pymorphy2
morph = pymorphy2.MorphAnalyzer()


def preprocessing(input_text, del_stopwords=False, del_digit=True):
    """
    :input: raw text
        1. lowercase, del punctuation, tokenize
        2. normal form
        3. del stopwords
        4. del digits
    :return: lemmas
    """
    if del_stopwords:
        russian_stopwords = set(stopwords.words('russian'))
    words = [x.lower().strip(string.punctuation+'»«–…') for x in word_tokenize(input_text)]
    lemmas = [morph.parse(x)[0].normal_form for x in words if x]

    lemmas_arr = []
    for lemma in lemmas:
        if del_stopwords:
            if lemma in russian_stopwords:
                continue
        if del_digit:
            if lemma.isdigit():
                continue
        lemmas_arr.append(lemma)
    return lemmas_arr

In [121]:
morph.parse('в')[0].normal_form

'в'

### Tf-Idf

In [176]:
def get_clear_texts(corpus, del_stopwords=True):
    for fn in corpus:
        with open(os.path.join("corpus", fn), 'r', encoding='utf-8') as f:
            text = '\n'.join(f.readlines()[3:])
            yield ' '.join(preprocessing(text, del_stopwords))

In [94]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfvect = TfidfVectorizer()
tf_matrix = tfidfvect.fit_transform(get_clear_texts(corpus)).T.toarray()


In [95]:
tf_matrix.shape

(883, 5)

In [96]:
import pandas as pd

matrix = pd.DataFrame(data=tf_matrix, index=tfidfvect.get_feature_names())

In [191]:
tfidf_result = []
for doc in matrix:
    print("Номер текста: ", doc)
    res = list(matrix[doc].nlargest(5).index)
    tfidf_result.append(set(res))
    print(res)

Номер текста:  0
['queen', 'произведение', 'all', 'is', 'jailhouse']
Номер текста:  1
['художник', 'русский', 'победа', 'это', 'человек']
Номер текста:  2
['scorpions', 'арен', 'втб', 'задержать', 'концерт']
Номер текста:  3
['выставка', 'искусство', 'eva', 'творчество', 'год']
Номер текста:  4
['миллениал', 'поколение', 'год', 'возраст', 'рост']


Кажется, что на более-менее длинных текстах неплохо работает - третий текст почти идеальный. Тем не менее, для такого маленького корпуса использовать этот способ довольно бессмысленно, так как на первый план выйдут просто уникальные для корпуса слова в тексте. Это заметно по первому тексту — его ключевые слова все в латинице, потому что в других текстах таких слов просто не было.

### RAKE

In [116]:
import RAKE

In [117]:
rake = RAKE.Rake(stopwords.words('russian'))

In [189]:
rake_result = []
for _id, text in enumerate(get_clear_texts(corpus, del_stopwords=False)):
    print("Номер текста: ", _id)
    result = rake.run(normalize_text_simple_word_tokenize(text), maxWords=3, minFrequency=1)
    rake_result.append({i[0] for i in result})
    print(sorted(result, key=lambda x: x[1], reverse=True)[:5])

Номер текста:  0
[('перечень музыкальный произведение', 9.0), ('великий отечественный война', 9.0), ('любить высоцкий марионетка', 9.0), ('культурный норматив школьник', 9.0), ('подрастать поколение макар', 9.0)]
Номер текста:  1
[('духовный основа который', 9.0), ('совместный сбережение насыщение', 9.0), ('глобальный смысл искусство', 8.5), ('единство наш победа', 8.5), ('это смысл живопись', 8.0)]
Номер текста:  2
[('московский стадион втб-арена', 14.5), ('слушать скорп дурацкий', 9.0), ('свой заявление умудриться', 9.0), ('вторник выступить', 4.0), ('добиться заголовок', 4.0)]
Номер текста:  3
[('фонд русский меценат', 9.0), ('широкий круг зритель', 9.0), ('оборонный ведомство девочка', 9.0), ('несколько месяц пошлый', 9.0), ('колония общий режим', 9.0)]
Номер текста:  4
[('рубеж тысячелетие доказать', 9.0), ('угроза общественный благополучие', 9.0), ('центр экономический рост', 9.0), ('ближний год стоимость', 9.0), ('год миллениал чаща', 9.0)]


Извлекаются так себе. Попробуем другой токенизатор?

In [139]:
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize
m = MorphAnalyzer()
def normalize_text_simple_word_tokenize(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        lemmas.append(
            m.parse(t)[0].normal_form
        )
    return ' '.join(lemmas)

In [143]:
texts = []
for fn in corpus:
        with open(os.path.join("corpus", fn), 'r', encoding='utf-8') as f:
            text = '\n'.join(f.readlines()[3:])
            texts.append(text)

In [157]:
for _id, text in enumerate(texts):
    print("Номер текста: ", _id)
    result = rake.run(normalize_text_simple_word_tokenize(text), maxWords=3, minFrequency=1)
    print(sorted(result, key=lambda x: x[1], reverse=True)[:5])

Номер текста:  0
[('перечень музыкальный произведение', 9.0), ('великий отечественный война', 9.0), ('культурный норматив школьник', 9.0), ('подрастать поколение макаре', 9.0), ('сегодняшний репутация подонок', 9.0)]
Номер текста:  1
[('интервью киреев газета', 9.0), ('научиться отличать добро', 9.0), ('глобальный смысл искусство', 8.5), ('стать этакий гуру', 8.5), ('учить человек жизнь', 8.5)]
Номер текста:  2
[('рамка мировой турне', 9.0), ('бессмысленно озвучивать остальной', 9.0), ('вторник выступить', 4.0), ('московский стадион', 4.0), ('втб-арена', 4.0)]
Номер текста:  3
[('пресс-релиз сказать', 9.0), ('широкий круг зритель', 9.0), ('высокостатусный таинственный незнакомка', 9.0), ('любимица минкульт рф', 9.0), ('самый знойный блондинка', 9.0)]
Номер текста:  4
[('чей становление прийтись', 9.0), ('угроза общественный благополучие', 9.0), ('копеечка система здравоохранение', 9.0), ('высокий уровень холестерин', 9.0), ('спектр поведенческий расстройство', 9.0)]


Стало хуже. Без лемматизации?

In [158]:
for _id, text in enumerate(texts):
    print("Номер текста: ", _id)
    result = rake.run(normalize_text_simple_word_tokenize(text), maxWords=3, minFrequency=1)
    print(sorted(result, key=lambda x: x[1], reverse=True)[:5])

Номер текста:  0
[('перечень музыкальный произведение', 9.0), ('великий отечественный война', 9.0), ('культурный норматив школьник', 9.0), ('подрастать поколение макаре', 9.0), ('сегодняшний репутация подонок', 9.0)]
Номер текста:  1
[('интервью киреев газета', 9.0), ('научиться отличать добро', 9.0), ('глобальный смысл искусство', 8.5), ('стать этакий гуру', 8.5), ('учить человек жизнь', 8.5)]
Номер текста:  2
[('рамка мировой турне', 9.0), ('бессмысленно озвучивать остальной', 9.0), ('вторник выступить', 4.0), ('московский стадион', 4.0), ('втб-арена', 4.0)]
Номер текста:  3
[('пресс-релиз сказать', 9.0), ('широкий круг зритель', 9.0), ('высокостатусный таинственный незнакомка', 9.0), ('любимица минкульт рф', 9.0), ('самый знойный блондинка', 9.0)]
Номер текста:  4
[('чей становление прийтись', 9.0), ('угроза общественный благополучие', 9.0), ('копеечка система здравоохранение', 9.0), ('высокий уровень холестерин', 9.0), ('спектр поведенческий расстройство', 9.0)]


Примерно так же плохо, как со вторым токенизатором. NLTK победил.

### Text Rank

In [161]:
from gensim.summarization import keywords as kw

In [188]:
textrank_result = []
for i, text in enumerate(get_clear_texts(corpus, del_stopwords=True)):
    print("Номер текста: ", i)
    result = kw(text, pos_filter=[], scores=True)[:5]
    textrank_result.append({i[0] for i in result})
    print(result)

Номер текста:  0
[('queen', 0.19715471893876976), ('перечень произведение', 0.15815450582858875), ('песнь', 0.14899931091617613), ('nirvana', 0.14011258213601038), ('мединскии', 0.13867923412570002)]
Номер текста:  1
[('русскии', 0.21301988402043467), ('это', 0.21034994525180167), ('человек', 0.16469527233613862), ('выставка художник владимир киреев', 0.1486620212945899), ('художественныи', 0.13744413763348381)]
Номер текста:  2
[('scorpions', 0.20424469698559147), ('арен задержать', 0.17803276044910793), ('московскии', 0.17345960559635792), ('втб', 0.1700592017740677), ('концерт группа', 0.15065155780430467)]
Номер текста:  3
[('год', 0.1713696625003155), ('москва', 0.1591951348039235), ('работа', 0.14725844670832733), ('современныи искусство', 0.13204851077891744), ('выставка око представить произведение художник', 0.11999371695158326)]
Номер текста:  4
[('поколение', 0.23582193879266028), ('год миллениал человек', 0.20379251240160068), ('экономическии', 0.17253197252580932), ('рост'

А стоп-слова-то почистились, откуда взялось это?

In [179]:
'это' in set(stopwords.words('russian'))

False

А нету в nltk такого стоп-слова.

## Оценка точности

In [197]:
import numpy as np

def precision(pred, true):
    try:
        p = len(pred & true) / len(pred)
    except ZeroDivisionError:
        p = 0
    return p
    
def recall(pred, true):
    try:
        r = len(pred & true) / len(true)
    except ZeroDivisionError:
        r = 0
    return r

def f_measure(p, r):
    if p == 0 or r == 0:
        return 0
    return 2 * p * r / (p + r) 

def estimate(all_pred, all_true):
    res = []
    for pred, true in zip(all_pred, all_true):
        p = precision(pred, true)
        r = recall(pred, true)
        res.append([p, r, f_measure(p, r)])
    return np.mean(np.array(res), axis=0)

In [198]:
# text_rank
estimate(textrank_result, all_true_their)

array([0.04      , 0.03333333, 0.03636364])

In [201]:
estimate(textrank_result, all_true_mine)

array([0.08      , 0.1       , 0.08636364])

In [199]:
#rake
estimate(rake_result, all_true_their)

array([0.02863636, 0.19166667, 0.04977926])

In [202]:
estimate(rake_result, all_true_mine)

array([0.02363636, 0.15666667, 0.04093919])

In [200]:
#tfidf
estimate(tfidf_result, all_true_their)

array([0.2       , 0.2       , 0.19234654])

In [203]:
estimate(tfidf_result, all_true_mine)

array([0.2       , 0.22888889, 0.20493506])

Все очень плохо. Видимо, я плохо подобрала корпус, увы.

### Бонус

Попробуем спасти положение, вытащив именные группы.

In [222]:
from nltk.parse import DependencyGraph

def get_trees(path):
    trees = []

    with open(path, encoding='utf-8') as f:
        parsed_sents = f.read().split('\n\n')[1:]

    for sent in parsed_sents:
        tree = [line for line in sent.split('\n') if line and line[0] != '#']
        trees.append('\n'.join(tree))
    
    return trees

_FILTER_RELS = ['punct', 'conj', 'parataxis']
def get_subtree(nodes, node):
    if not nodes[node]['deps']:
        return [node]
    else:
        return [node] + [get_subtree(nodes, dep) for rel in nodes[node]['deps'] 
                         if rel not in _FILTER_RELS
                         for dep in nodes[node]['deps'][rel]]

In [223]:
def flatten(l):
    flat = []
    for el in l:
        if not isinstance(el, list):
            flat.append(el)
        else:
            flat += flatten(el)
    return flat

In [219]:
def get_nps(trees):
    result = []
    for t in trees:
        np_list = []
        g = DependencyGraph(t, top_relation_label='root')
        for n in g.nodes:
            if g.nodes[n]['ctag'] == 'NOUN':
                np = list(sorted(flatten(get_subtree(g.nodes, n))))
                np_list.append(
                    ' '.join([g.nodes[i]['word'] for i in np])
                )
        result.append(np_list)
    return result

In [224]:
conllu_corpus = [i for i in os.listdir("corpus") if i.endswith('.conllu')]
forest = []
corp_nps = 
for conll in conllu_corpus:
    trees = get_trees(os.path.join("corpus", conll))
    npr = get_nps(trees)

In [226]:
print(get_nps(trees))

[['Министерство культуры Российской Федерации', 'культуры Российской Федерации', 'перечень произведений по литературе', 'произведений по литературе', 'по литературе', 'изобразительному искусству', 'кинематографу', 'и музыке с которыми рекомендуется ознакомиться школьникам', 'школьникам'], ['В перечень музыкальных произведений с которыми надо ознакомиться учащимся 9 11 классов', 'музыкальных произведений с которыми надо ознакомиться учащимся 9 11 классов', '9 11 классов', 'хиты Queen Bohemian rhapsody'], ['песни Дунаевского из кинофильма Весёлые ребята', 'из кинофильма Весёлые ребята', 'Весёлые ребята', 'песни о Великой Отечественной войне', 'о Великой Отечественной войне', 'Марионетки Машины времени', 'и Солнечный остров', 'Машины времени', 'времени', 'Под небом голубым Гребенщикова', 'Перемен группы Кино Наутилуса Помпилиуса', 'группы Кино Наутилуса Помпилиуса', 'и Скованные одной цепью'], ['Личные музыкальные вкусы Владимира Мединского заместитель Мединского отвечающая за Культурный 